# Введение

Наша цель - построить математическую модель предсказания колебания биржевого курса для дальнейшего использования в качестве сигнала для краткосрочной биржевой торговли.

Для этого я выбрал валютную пару USD-EUR на рынке Форекс. В качестве площадки был выбран Альфа-Банк.

Конечно, написание коммерчески-успешного торгового автомата передо мной не стоит. Краткосрочная торговля - одна из самых сложных для предсказания видов биржевой торговли. Слишком много факторов влиет, которые в принципе не могут быть считаны моделью. Однако подобная работа поможет мне лучше разобраться и с работами моделей, и с тонкостями работы биржевых автоматов.

А самое главное - тестируя модель в продакте, что-бы увидеть результаты - не надо неделями и месяцами ждать, что-бы собрать достаточную статистику. Достаточно пары часов, а то и нескольких минут для получения результата

## 0. Первичная инициализация

Импорт библиотек, задание переменных, констант.

In [1]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

plt.style.use('seaborn-v0_8') #стиль отрисовки seaborn
%matplotlib inline

import MetaTrader5 as mt5 # для работы с торговой платформой

#from sklearn import model_selection #методы разделения и валидации
#from sklearn import linear_model #линейные модели
#from sklearn import metrics #метрики
#from sklearn import linear_model #линейные модели
#from sklearn import tree #деревья решений

#from sklearn import preprocessing
#from sklearn import ensemble #ансамбли

import pickle # сохранение объектов

from datetime import datetime

from tqdm import tqdm  # для отображения прогресса долгих рассчетов
tqdm.pandas()

import winsound  # пищим динамиком, когда надо
import time

In [2]:
# функция, что-бы попищать динамиком. Полезно, что-бы просигналить, что долгий рассчёт окончен

def beep():
    frequency = 2500  # Set Frequency To 2500 Hertz
    duration = 500  # Set Duration To 1000 ms == 1 second
    winsound.Beep(frequency, duration)
    winsound.Beep(round(frequency/2), duration)
    winsound.Beep(round(frequency/4), duration)
    winsound.Beep(round(frequency/2), duration)
    winsound.Beep(frequency, duration)

# 1. Анализ и обработка данных

## 1.1. Обработка и очистка данных

### 1.1.1. Получение датасета

Поскольку результаты, полученные моделью предполагается использовать в работе торгового терминала, логично, что датасет для работы лучше всего получить с этого-же терминала.

Терминал (Metatrader 5) уже должен быть установлен на машине, настроен и подключен к торговому счёту

Демо-счёт MT5 (с хеджированием)<br>
Логин:<br>
2000061156<br>
Пароль трейдера:<br>
R1b9j4d4g68L71q0  # пароль удалён, конечно

In [3]:
# выведем данные о пакете MetaTrader5
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.45


In [4]:
# установим подключение к терминалу MetaTrader 5 на указанный торговый счет
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [5]:
# получим данные о нашем выбранном инструменте

symbol = "EURUSDrfd"
symbol_info = mt5.symbol_info(symbol)

symbol_info

SymbolInfo(custom=False, chart_mode=0, select=True, visible=True, session_deals=0, session_buy_orders=0, session_sell_orders=0, volume=0, volumehigh=0, volumelow=0, time=1700043131, digits=5, spread=14, spread_float=True, ticks_bookdepth=0, trade_calc_mode=0, trade_mode=4, start_time=0, expiration_time=0, trade_stops_level=7, trade_freeze_level=0, trade_exemode=2, swap_mode=1, swap_rollover3days=3, margin_hedged_use_leg=False, expiration_mode=11, filling_mode=1, order_mode=55, order_gtc_mode=0, option_mode=0, option_right=0, bid=1.08673, bidhigh=1.08832, bidlow=1.08648, ask=1.08687, askhigh=1.08846, asklow=1.08662, last=0.0, lasthigh=0.0, lastlow=0.0, volume_real=0.0, volumehigh_real=0.0, volumelow_real=0.0, option_strike=0.0, point=1e-05, trade_tick_value=1.0, trade_tick_value_profit=1.0, trade_tick_value_loss=1.0, trade_tick_size=1e-05, trade_contract_size=100000.0, trade_accrued_interest=0.0, trade_face_value=0.0, trade_liquidity_rate=0.0, volume_min=0.01, volume_max=100.0, volume_s

Пока неизвестно, сколько нам потребуется данных для работы модели. Может, хватит данных за одну неделю. А, возможно, потребуется информация за несколько лет. Для начала используем данные за один месяц. В дальнейшем - проведем иследование, сколько данных оптимально нужно для обучения модели.

In [7]:
# получим все тики по выбранному инструменту за октябрь 2023 года

# создадим структуру DataFrame для хранения тиков
ticks_frame = pd.DataFrame(columns=['time', 'bid', 'ask', 'last', 'volume'])

# получим тиковые данные за 2020.10.01
#ticks = mt5.copy_ticks_from(symbol, datetime(2020,10,1,0,0,0,0), 100000, mt5.COPY_TICKS_ALL)
ticks = mt5.copy_ticks_range(symbol, datetime(2023,10,1,0,0,0,0), datetime(2023,10,31,0,0,0,0), mt5.COPY_TICKS_ALL)

# заполним DataFrame
ticks_frame = pd.DataFrame(ticks)

# выведем данные
ticks_frame

,time,bid,ask,last,volume,time_msc,flags,volume_real
0,1696212000,1.05589,1.05604,0.0,0,1696212000222,230,0.0
1,1696212000,1.05590,1.05604,0.0,0,1696212000494,226,0.0
2,1696212000,1.05589,1.05604,0.0,0,1696212000688,226,0.0
3,1696212000,1.05590,1.05604,0.0,0,1696212000725,226,0.0
4,1696212000,1.05590,1.05604,0.0,0,1696212000947,96,0.0
...,...,...,...,...,...,...,...,...
2574616,1698699591,1.06107,1.06121,0.0,0,1698699591590,230,0.0
2574617,1698699591,1.06108,1.06122,0.0,0,1698699591703,230,0.0
2574618,1698699592,1.06107,1.06121,0.0,0,1698699592087,230,0.0
2574619,1698699598,1.06106,1.06120,0.0,0,1698699598075,230,0.0


### 1.1.2. Очистка данных

In [8]:
# Выведем информацию о DataFrame
ticks_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2574621 entries, 0 to 2574620
Data columns (total 8 columns):
 #   Column       Dtype  
---  ------       -----  
 0   time         int64  
 1   bid          float64
 2   ask          float64
 3   last         float64
 4   volume       uint64 
 5   time_msc     int64  
 6   flags        uint32 
 7   volume_real  float64
dtypes: float64(4), int64(2), uint32(1), uint64(1)
memory usage: 147.3 MB


In [9]:
# выведем статистику по тиковым данным
ticks_frame.describe()

,time,bid,ask,last,volume,time_msc,flags,volume_real
count,2.574621e+06,2.574621e+06,2.574621e+06,2574621.0,2574621.0,2.574621e+06,2.574621e+06,2574621.0
mean,1.697351e+09,1.055751e+00,1.055893e+00,0.0,0.0,1.697351e+12,2.177882e+02,0.0
std,7.186076e+05,4.358614e-03,4.358339e-03,0.0,0.0,7.186076e+08,3.866391e+01,0.0
min,1.696212e+09,1.044760e+00,1.044900e+00,0.0,0.0,1.696212e+12,4.000000e+00,0.0
25%,1.696620e+09,1.053120e+00,1.053260e+00,0.0,0.0,1.696620e+12,2.300000e+02,0.0
50%,1.697226e+09,1.055790e+00,1.055940e+00,0.0,0.0,1.697226e+12,2.300000e+02,0.0
75%,1.698056e+09,1.058800e+00,1.058940e+00,0.0,0.0,1.698056e+12,2.300000e+02,0.0
max,1.698700e+09,1.069370e+00,1.069510e+00,0.0,0.0,1.698700e+12,2.300000e+02,0.0


Как видим - поля last, volume и volume_real - пустые. Их можно удалить